In [5]:
import pandas as pd
import numpy as np
import io
import math
import random
import matplotlib.pyplot as plt
import librosa
import librosa.display
import cv2
import cmapy
import nlpaug
import nlpaug.augmenter.audio as naa
from scipy.signal import butter, lfilter
import torch
from tqdm import tqdm
import os

In [6]:
audio_train=pd.read_csv('Temp_train_audio.csv')
audio_train.head(4)

,Unnamed: 0,Patient_ID,Disease,Recording_index,Chest_Location,Acquisition_Mode,Recording_equipment,Respiratory_cycles,Normal_cycles,Wheeze_cycles,Crackle_cycles,Both,Filename,Train/Test
0,13,107,NaN,2b3,Al,mc,AKGC417L,8,1,0,7,0,107_2b3_Al_mc_AKGC417L.txt,train
1,14,107,NaN,2b3,Ar,mc,AKGC417L,8,0,0,1,7,107_2b3_Ar_mc_AKGC417L.txt,train
2,15,107,NaN,2b3,Ll,mc,AKGC417L,8,0,0,8,0,107_2b3_Ll_mc_AKGC417L.txt,train
3,16,107,NaN,2b3,Lr,mc,AKGC417L,8,1,0,7,0,107_2b3_Lr_mc_AKGC417L.txt,train


In [7]:
sample_rate=4000
fs=30000
data_dir='AudioFiles2/'
desired_length=8
n_mels = 64
nfft = 256
hop = nfft//2
f_max = 2000
train_flag=1
lowcut=3000
highcut=5000

In [8]:
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a

def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b,a, data)
    return y

In [9]:
#slicing into breath cycles
def slice_data(start, end, raw_data, sample_rate):
    max_ind = len(raw_data) 
    start_ind = min(int(start * sample_rate), max_ind)
    end_ind = min(int(end * sample_rate), max_ind)
    y=butter_bandpass_filter(raw_data[start_ind: end_ind], lowcut, highcut, fs, order=5)
    return y

In [10]:
#getting label of each cycles
def get_label(crackle, wheeze):
    if crackle == 0 and wheeze == 0:
        return 0
    elif crackle == 1 and wheeze == 0:
        return 1
    elif crackle == 0 and wheeze == 1:
        return 2
    else:
        return 3

In [11]:
#getting breath cycle samples
def get_sound_samples(file_name, data_dir, sample_rate):
    sample_data = [file_name]
    data, rate = librosa.load(os.path.join(data_dir, file_name+'.wav'), sr=sample_rate)
    recording_annotations=pd.read_csv(data_dir+file_name+'.txt', sep='\t', header=None)
    
    for i in range(len(recording_annotations.index)):
        row = recording_annotations.loc[i]
        start = row[0]
        end = row[1]
        crackles = row[2]
        wheezes = row[3]
        audio_chunk = slice_data(start, end, data, rate)
        sample_data.append((audio_chunk, start,end, get_label(crackles, wheezes)))
    return sample_data

In [12]:
#creating list of breath cycles with respective labels
filenames=audio_train['Filename']
filenames_with_labels=[]
cycle_list = []
classwise_cycle_list = [[], [], [], []]
for idx, filename in tqdm(enumerate(filenames)):
            file_name=filename.split('.')[0]
            print(file_name+" going on "+str(idx)+" out of "+str(len(filenames)))
            data = get_sound_samples(file_name, data_dir, sample_rate)
            cycles_with_labels = [(d[0], d[3], file_name, cycle_idx, 0) for cycle_idx, d in enumerate(data[1:])]
            cycle_list.extend(cycles_with_labels)
            for cycle_idx, d in enumerate(cycles_with_labels):
                filenames_with_labels.append(file_name+'_'+str(d[3])+'_'+str(d[1]))
                classwise_cycle_list[d[1]].append(d)

0it [00:00, ?it/s]

107_2b3_Al_mc_AKGC417L going on 0 out of 41


1it [00:03,  3.11s/it]

107_2b3_Ar_mc_AKGC417L going on 1 out of 41


2it [00:03,  1.65s/it]

...

110_1b1_Pr_sc_Meditron going on 29 out of 41
110_1p1_Al_sc_Meditron going on 30 out of 41
110_1p1_Ll_sc_Meditron going on 31 out of 41
110_1p1_Lr_sc_Meditron going on 32 out of 41
110_1p1_Pr_sc_Meditron going on 33 out of 41
111_1b2_Tc_sc_Meditron going on 34 out of 41


35it [00:22,  3.69it/s]

111_1b3_Tc_sc_Meditron going on 35 out of 41


41it [00:23,  1.73it/s]

112_1b1_Ar_sc_Meditron going on 36 out of 41
112_1b1_Lr_sc_Meditron going on 37 out of 41
112_1p1_Ll_sc_Litt3200 going on 38 out of 41
112_1p1_Pl_sc_Litt3200 going on 39 out of 41
112_1p1_Pr_sc_Litt3200 going on 40 out of 41


In [13]:
#splitting and padding cycles
def split_and_pad(original, desiredLength, sample_rate, types=0):
	if types==0:
		return split_and_pad_old(original, desiredLength, sample_rate)

	output_buffer_length = int(desiredLength*sample_rate)
	soundclip = original[0].copy()
	n_samples = len(soundclip)

	output = []
	if n_samples > output_buffer_length:
		frames = librosa.util.frame(soundclip, frame_length=output_buffer_length, hop_length=output_buffer_length//2, axis=0)
		for i in range(frames.shape[0]):
			output.append((frames[i], original[1], original[2], original[3], original[4], i, 0))

		last_id = frames.shape[0]*(output_buffer_length//2)
		last_sample = soundclip[last_id:]; pad_times = (output_buffer_length-len(last_sample))/len(last_sample)
		padded = generate_padded_samples(soundclip, last_sample, output_buffer_length, sample_rate, types)
		output.append((padded, original[1], original[2], original[3], original[4], i+1, pad_times))

	else:
		padded = generate_padded_samples(soundclip, soundclip, output_buffer_length, sample_rate, types); pad_times = (output_buffer_length-len(soundclip))/len(soundclip)
		output.append((padded, original[1], original[2], original[3], original[4], 0, pad_times))

	return output


def split_and_pad_old(original, desiredLength, sample_rate):
    output_buffer_length = int(desiredLength * sample_rate)
    soundclip = original[0].copy()
    n_samples = len(soundclip)
    total_length = n_samples / sample_rate #length of cycle in seconds
    n_slices = int(math.ceil(total_length / desiredLength)) #get the minimum number of slices needed
    samples_per_slice = n_samples // n_slices
    src_start = 0 #Staring index of the samples to copy from the original buffer
    output = [] #Holds the resultant slices
    for i in range(n_slices):
        src_end = min(src_start + samples_per_slice, n_samples)
        length = src_end - src_start
        copy = generate_padded_samples_old(soundclip[src_start:src_end], output_buffer_length)
        output.append((copy, original[1], original[2], original[3], original[4], i))
        src_start += length
    return output


def generate_padded_samples(original, source, output_length, sample_rate, types):
	copy = np.zeros(output_length, dtype=np.float32)
	src_length = len(source)
	left = output_length-src_length # amount to be padded
	# pad front or back
	prob = random.random()
	if types == 1:
		aug = original
	else:
		aug = gen_augmented(original, sample_rate)

	while len(aug) < left:
		aug = np.concatenate([aug, aug])

	if prob < 0.5:
		#pad back
		copy[left:] = source
		copy[:left] = aug[len(aug)-left:]
	else:
		#pad front
		copy[:src_length] = source[:]
		copy[src_length:] = aug[:left]

	return copy


In [14]:
#General augmentation augmentation
def gen_augmented(original, sample_rate):
	# list of augmentors available from the nlpaug library
	augment_list = [
	#naa.CropAug(sampling_rate=sample_rate)
	naa.NoiseAug(),
	naa.SpeedAug(),
	naa.LoudnessAug(factor=(0.5, 2)),
	naa.VtlpAug(sampling_rate=sample_rate, zone=(0.0, 1.0)),
	naa.PitchAug(sampling_rate=sample_rate, factor=(-1,3))
	]
	# sample augmentation randomly
	aug_idx = random.randint(0, len(augment_list)-1)
	augmented_data = augment_list[aug_idx].augment(original)
	return augmented_data

In [15]:
#creating sliced and padded data = audio_data
audio_data=[]
for idx, sample in enumerate(cycle_list):
            print(str(idx)+" out of "+str(len(cycle_list)))
            output = split_and_pad(sample, desired_length, sample_rate, types=1)
            audio_data.extend(output)

0 out of 365
1 out of 365
2 out of 365
3 out of 365
4 out of 365
5 out of 365
6 out of 365
7 out of 365
8 out of 365
9 out of 365
10 out of 365
11 out of 365
12 out of 365
13 out of 365
14 out of 365
15 out of 365
16 out of 365
17 out of 365
18 out of 365
19 out of 365
20 out of 365
21 out of 365
22 out of 365
23 out of 365
24 out of 365
25 out of 365
26 out of 365
27 out of 365
28 out of 365
29 out of 365
30 out of 365
31 out of 365
32 out of 365
33 out of 365
34 out of 365
35 out of 365
36 out of 365
37 out of 365
38 out of 365
39 out of 365
40 out of 365
41 out of 365
42 out of 365
43 out of 365
44 out of 365
45 out of 365
46 out of 365
47 out of 365
48 out of 365
49 out of 365
50 out of 365
51 out of 365
52 out of 365
53 out of 365
54 out of 365
55 out of 365
56 out of 365
57 out of 365
58 out of 365
59 out of 365
60 out of 365
61 out of 365
62 out of 365
63 out of 365
64 out of 365
65 out of 365
66 out of 365
67 out of 365
68 out of 365
69 out of 365
70 out of 365
71 out of 365
72

In [16]:
#Device_wise_list
file_to_device = {}
device_to_id = {}
device_id = 0
files = audio_train['Filename']#os.listdir(data_dir)
device_patient_list = []
pats = []
for f in files:
    device = f.strip().split('_')[-1].split('.')[0]
    if device not in device_to_id:
        device_to_id[device] = device_id
        device_id += 1
        device_patient_list.append([])
    file_to_device[f.strip().split('.')[0]] = device_to_id[device]
    pat = f.strip().split('_')[0]
    if pat not in device_patient_list[device_to_id[device]]:
        device_patient_list[device_to_id[device]].append(pat)
    if pat not in pats:
        pats.append(pat)

In [17]:
#creating labels for sliced and paddedd data = audio_data
class_probs=np.zeros(4)
identifiers=[]
device_wise=[]
labels=[]
for idx in range(device_id):
            device_wise.append([])
for idx, sample in enumerate(audio_data):
            class_probs[sample[1]] += 1.0
            labels.append(sample[1])
            identifiers.append(sample[2]+'_'+str(sample[3])+'_'+str(sample[1]))
            device_wise[file_to_device[sample[2]]].append(sample)

<b>audio_data</b> is the data which padded and augmented (in the simple way)

In [18]:
#creating mel spectrograms from augmented and padded data
def create_mel_raw(current_window, sample_rate, n_mels=128, f_min=50, f_max=4000, nfft=2048, hop=512, resz=1):
    S = librosa.feature.melspectrogram(y=current_window, sr=sample_rate, n_mels=n_mels, fmin=f_min, fmax=f_max, n_fft=nfft, hop_length=hop)
    S = librosa.power_to_db(S, ref=np.max)
    S = (S-S.min()) / (S.max() - S.min())
    S *= 255
    img = cv2.applyColorMap(S.astype(np.uint8), cmapy.cmap('magma'))
    height, width, _ = img.shape
    if resz > 0:
        img = cv2.resize(img, (width*resz, height*resz), interpolation=cv2.INTER_LINEAR)
    img = cv2.flip(img, 0)
    return img

In [19]:
train_dataset=[]
for index in range(len(audio_data)):
    audio=audio_data[index][0]
    # convert audio signal to spectrogram
    # spectrograms resized to 3x of original size
    audio_image = cv2.cvtColor(create_mel_raw(audio, sample_rate, f_max=f_max, 
            n_mels=n_mels, nfft=nfft, hop=hop, resz=3), cv2.COLOR_BGR2RGB)
    
    # blank region clipping
    audio_raw_gray = cv2.cvtColor(create_mel_raw(audio, sample_rate, f_max=f_max, 
            n_mels=n_mels, nfft=nfft, hop=hop), cv2.COLOR_BGR2GRAY)
    audio_raw_gray[audio_raw_gray < 10] = 0
    for row in range(audio_raw_gray.shape[0]):
        black_percent = len(np.where(audio_raw_gray[row,:]==0)[0])/len(audio_raw_gray[row,:])
        if black_percent < 0.80:
            break   
    if (row+1)*3 < audio_image.shape[0]:
        audio_image = audio_image[(row+1)*3:, :, :]
    audio_image = cv2.resize(audio_image, (audio_image.shape[1], n_mels*3), interpolation=cv2.INTER_LINEAR)
        
    
    label = audio_data[index][1]
    train_dataset.append((audio_image, label))

In [20]:
train_dataset[0][0].shape

(192, 753, 3)

In [21]:
X_train=[]
y_train=[]
for i in range(len(train_dataset)):
    X_train.append(train_dataset[i][0])
    y_train.append(train_dataset[i][1])  
#display(X_train)
np.save('X_train_temp', np.concatenate(X_train, axis=0).reshape(365, 192, 753, 3))
np.save('y_train_temp', y_train)

# print("X_train length : "+str(len(X_train)))
# print("X_train sample length : "+str(X_train[0].shape))
# print("Array : "+str(np.concatenate(X_train, axis=0).shape))

# np.concatenate(X_train, axis=0).reshape(365, 192, 753, 3).shape